In [1]:
import pandas as pd
import praw

In [2]:
from praw.models import MoreComments
import numpy as np

In [3]:
reddit = praw.Reddit(user_agent = "Scraper", client_id = "NuVXna0yymo8foTphxFwAQ",
                     client_secret = "mQAuSnpOKTYAKqqtlbzZdF5Ax9m9Qg")

In [4]:
comments = []
titles = []
for submission in reddit.subreddit("nba").search("Tatum", sort = "top", time_filter = "month"):
    if "Tatum" in submission.title:
        submission.comments.replace_more(limit = 0)
        if (len(comments) < 1000):
            for i in range(len(submission.comments)):
                comments.append(submission.comments[i].body)

In [5]:
print(len(comments))

1079


In [6]:
comments_df = pd.DataFrame(comments)

In [7]:
comments_df.rename(columns = {0: "comment"}, inplace = True)

In [8]:
text = "I am a bot" ##remove automated comments
comments_df[comments_df['comment'].str.contains(text)] 

,comment
0,"**Replays/alternate angles**\n\n\n*I am a bot,..."
189,"**Replays/alternate angles**\n\n\n*I am a bot,..."


In [9]:
comments_df.drop([0, 189], inplace = True)

In [10]:
comments_df.head()

,comment
1,"Including Tatum here is disgraceful, Haliburto..."
2,Hali was cooking from the bench. 300 Daps 40 c...
3,Hali at least looks happy lol
4,"LMAO, sadly that’s what happens when Book/Jrue..."
5,how tyrese cooking him on the bench


In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification



In [12]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/opt/anaconda3/envs/sentimentAnalysis/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [41]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest", max_length = 512, truncation = True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
data = list(comments_df['comment'])
sentiments = pipe(data)

In [43]:
sentiments

[{'label': 'negative', 'score': 0.9181158542633057},
 {'label': 'positive', 'score': 0.970832884311676},
 {'label': 'positive', 'score': 0.9021363854408264},
 {'label': 'negative', 'score': 0.4951000511646271},
 {'label': 'neutral', 'score': 0.6659390330314636},
 {'label': 'negative', 'score': 0.7037053108215332},
 {'label': 'negative', 'score': 0.865164041519165},
 {'label': 'positive', 'score': 0.5602708458900452},
 {'label': 'negative', 'score': 0.9140520691871643},
 {'label': 'neutral', 'score': 0.7276932001113892},
 {'label': 'positive', 'score': 0.9644150137901306},
 {'label': 'negative', 'score': 0.556774377822876},
 {'label': 'positive', 'score': 0.8160250782966614},
 {'label': 'neutral', 'score': 0.8479279279708862},
 {'label': 'neutral', 'score': 0.4151810109615326},
 {'label': 'positive', 'score': 0.962120771408081},
 {'label': 'neutral', 'score': 0.4593324363231659},
 {'label': 'negative', 'score': 0.9197065234184265},
 {'label': 'neutral', 'score': 0.8043766021728516},
 {'

In [45]:
labels = []
scores = []
for dict in sentiments:
    labels.append(dict['label'])
    scores.append(dict['score'])
    

In [47]:
comments_df['labels'] = labels
comments_df['scores'] = scores

In [48]:
comments_df.head()

,comment,labels,scores
1,"Including Tatum here is disgraceful, Haliburto...",negative,0.918116
2,Hali was cooking from the bench. 300 Daps 40 c...,positive,0.970833
3,Hali at least looks happy lol,positive,0.902136
4,"LMAO, sadly that’s what happens when Book/Jrue...",negative,0.495100
5,how tyrese cooking him on the bench,neutral,0.665939


In [51]:
##write it to external csv file
comments_df.to_csv("labeled_comments.csv")